In [1]:
import os
%pwd

'/home/gautham/E2EProjects/TextSummarisation/research'

In [2]:
os.chdir("../")

In [3]:
! pwd

/home/gautham/E2EProjects/TextSummarisation


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir : Path
    data_path : Path
    model_ckpt : str
    num_train_epochs : int
    warmup_steps : int
    per_device_train_batch_size : int
    weight_decay : float
    logging_steps : int
    evaluation_strategy : str
    eval_steps : int
    save_steps : float
    gradient_accumulation_steps : int

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


class ConfigarationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 param_file_path = PARAMS_FILE_FILE):
        

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(param_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=1e6,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig) -> None:
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_pegasus)

        # load data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(output_dir=self.config.root_dir,
                                         num_train_epochs=self.config.num_train_epochs,
                                         warmup_steps=self.config.warmup_steps,
                                         per_device_train_batch_size=self.config.per_device_train_batch_size,
                                         weight_decay=self.config.weight_decay,
                                         logging_steps=self.config.logging_steps,
                                         evaluation_strategy=self.config.evaluation_strategy,
                                         eval_steps=self.config.eval_steps,
                                         save_steps=self.config.save_steps,
                                         gradient_accumulation_steps=self.config.gradient_accumulation_steps)
        
        trainer = Trainer(model= model_pegasus, 
                          args=trainer_args,
                          train_dataset=dataset_samsum_pt["train"],
                          eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        # save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [9]:
try:
    config = ConfigarationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as exception:
    raise exception

[2024-04-06 17:07:50,910: INFO: common: yaml file : config/config.yaml loded successfully]
[2024-04-06 17:07:50,940: INFO: common: yaml file : params.yaml loded successfully]
[2024-04-06 17:07:50,947: INFO: common: Created directory at : artifacts]
[2024-04-06 17:07:50,952: INFO: common: Created directory at : artifacts/model_trainer]


: 